In [8]:
import mysql.connector
import psycopg2

In [9]:
def source():
    mydb = mysql.connector.connect(
      host='localhost',
      user='root',
      password='root',
      database='ecommerce'
    )
    
    return mydb

In [10]:
def dest():
    connection = psycopg2.connect(user="warehouse",
                                  password="warehouse",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="warehouse")
    return connection
    

In [11]:
def get_last_product_id(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("select product_id from products order by product_id DESC limit 1")
        return cursor.fetchone()[0]
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [12]:
def get_last_category_id(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("select category_id from categories order by category_id DESC limit 1")
        return cursor.fetchone()[0]
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [35]:
def get_last_order_id(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("select order_id from sales order by order_id DESC limit 1")
        res = cursor.fetchone()
        return res[0] if res != None else 0
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [36]:
last_product_id = get_last_product_id(dest())
last_category_id = get_last_category_id(dest())
last_order_id = get_last_order_id(dest())

PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed


In [37]:
last_product_id, last_category_id, last_order_id

(21, 11, 0)

In [41]:
def get_missing_products(db, last_product_id):
    cursor = db.cursor()
    print("last prod id {}".format(last_product_id))

    sql = '''
        SELECT
            p.id as product_id,
            p.code as name
        FROM
            ecommerce_product p 
        WHERE
            p.id > %s
    ;
    '''

    cursor.execute(sql, (last_product_id,))
    
    results = cursor.fetchall()
    return results
    

In [42]:
def get_missing_categories(db, last_category_id):
    cursor = db.cursor()
    print("last cat id {}".format(last_category_id))

    sql = '''
        SELECT
            c.id as category_id,
            c.code as name
        FROM
            ecommerce_taxon c 
        WHERE
            c.id > %s
    ;
    '''

    cursor.execute(sql, (last_category_id,))
    
    results = cursor.fetchall()
    return results

In [45]:
def get_missing_sales(db, last_order_id):
    cursor = db.cursor()
    print("last order id {}".format(last_order_id))

    sql = '''
        SELECT
            oi.order_id as order_id,
            pv.product_id as product_id,
            p.main_taxon_id as category_id,
            sum(oi.total) as total_sale
        FROM
            ecommerce_order_item oi
        JOIN 
            ecommerce_product_variant pv on pv.id = oi.variant_id
        JOIN 
            ecommerce_product p on pv.product_id = p.id
        WHERE
            oi.order_id > %s
        GROUP BY
            oi.order_id,
            pv.product_id
    ;
    '''

    cursor.execute(sql, (last_order_id,))
    
    results = cursor.fetchall()
    return results

In [46]:
products = get_missing_products(source(), get_last_product_id(dest()))
categories = get_missing_categories(source(), get_last_category_id(dest()))
sales = get_missing_sales(source(), get_last_order_id(dest()))

PostgreSQL connection is closed
last prod id 21
PostgreSQL connection is closed
last cat id 11
PostgreSQL connection is closed
last order id 0


In [53]:
from psycopg2.extras import execute_values
def insert_many_dest(connection, sql, items):
    try:
        cursor = connection.cursor()
        execute_values(cursor, sql, items)
        connection.commit()
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [54]:
products = get_missing_products(source(), get_last_product_id(dest()))
insert_many_dest(dest(), 'Insert into products (product_id, name) VALUES %s', products)

PostgreSQL connection is closed
last prod id 21
PostgreSQL connection is closed


In [55]:
def extract():
    products = get_missing_products(source(), get_last_product_id(dest()))
    categories = get_missing_categories(source(), get_last_category_id(dest()))
    sales = get_missing_sales(source(), get_last_order_id(dest()))
    
    return (products, categories, sales)

In [56]:
def transform(data):
    return data

In [68]:
def load(data):
    (products, categories, sales) = data
    insert_many_dest(dest(), 'Insert into products (product_id, name) VALUES %s', products)
    insert_many_dest(dest(), 'Insert into categories (category_id, name) VALUES %s', categories)
    insert_many_dest(dest(), 'Insert into sales (order_id, product_id, category_id, total_sales) VALUES %s', sales)

In [78]:
load(transform(extract()))


PostgreSQL connection is closed
last prod id 21
PostgreSQL connection is closed
last cat id 11
PostgreSQL connection is closed
last order id 1232
PostgreSQL connection is closed
PostgreSQL connection is closed
PostgreSQL connection is closed
